In [1]:
# conda list
import glob

In [2]:
from google.cloud import storage, bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])

In [3]:
storage_client = storage.Client(credentials=credentials, project=credentials.project_id)
ssd_bucket = 'scanny-data-backup'
bucket = storage_client.get_bucket(ssd_bucket)

In [4]:
for obj in bucket.list_blobs(prefix=None):
    print(obj.name)

Backupdata/Box Count W49-W13.xlsx
Backupdata/Recipe Detail W49 - W13.xlsx
Backupdata/Scanny Data W49-W13.xlsx
backup_by_weeks/
backup_by_weeks/Data/Box Count.csv
backup_by_weeks/Data/Main.csv
backup_by_weeks/Data/Recipe Details.csv
backup_by_weeks/Data/W06/Kitting Dashboard -- W06.pdf
backup_by_weeks/Data/W06/W06.xlsx
backup_by_weeks/Data/W06/Week06Kitting.pdf
backup_by_weeks/Data/W06/prepared.csv
backup_by_weeks/Data/W06/raw.xlsx
backup_by_weeks/Data/W06/staged.csv
backup_by_weeks/Data/W07/Kitting Dasboard -- W07.pdf
backup_by_weeks/Data/W07/W07.xlsx
backup_by_weeks/Data/W07/W07kitting.pdf
backup_by_weeks/Data/W07/prepared.csv
backup_by_weeks/Data/W07/raw.xlsx
backup_by_weeks/Data/W07/staged.csv
backup_by_weeks/Data/W08/Kitting KPI Dashboard - W08.pdf
backup_by_weeks/Data/W08/W08 Assembly.xlsx
backup_by_weeks/Data/W08/W08.xlsx
backup_by_weeks/Data/W08/prepared.csv
backup_by_weeks/Data/W08/raw.xlsx
backup_by_weeks/Data/W08/staged.csv
backup_by_weeks/Data/W09/W09 Assembly.xlsx
backup_by

In [5]:
def download_from_gcs(gcs_folder, local_path):
    """
    gcs_folder -- the blob storage of the weeks' data
    local_path -- point to the desginated local folder 
    """
    for obj in bucket.list_blobs(prefix = gcs_folder):
        fileName = obj.name.split('/')[-1]
        obj.download_to_filename(os.path.join(local_path,fileName))

In [6]:
# data_week_folder = 'backup_by_weeks/Data/W29'
# local_folder = r"C:\Users\Peiran Quan\Desktop"
# download_from_gcs(data_week_folder, local_folder)